In [194]:
## MLFLOW_TRACKING_URI='https://qualiextra-project-6b29e931dd67.herokuapp.com/'

In [195]:
import pandas as pd
import datetime
import numpy as np
from plotly.subplots import make_subplots
from prophet import Prophet

import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,  mean_absolute_error, mean_absolute_percentage_error


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier

In [196]:
data = pd.read_csv("../Missions.csv", sep=';')

In [197]:
mask = data["hôtel"].notna()
data = data[mask]
data["date_debut"] = data.apply(lambda x: x["date"].split(" →")[0], axis=1)
data["date_debut"] = data["date_debut"].apply(
    lambda x: x.replace(" (UTC+3)", "")
)
data["date_debut"] = data["date_debut"].apply(lambda x: x.replace(" (UTC)", ""))
data["date_debut"] = pd.to_datetime(data["date_debut"], format="%d/%m/%Y %H:%M")
data["date"] = pd.to_datetime(data["date_debut"]).dt.date
data["time_delta"] = data["nbre d'heures"].apply(lambda x: pd.to_timedelta(x))
data["date_fin"] = data.apply(
    lambda x: x["date_debut"] + x["time_delta"], axis=1
)
data["Propriété"] = data.apply(
    lambda x: x["hôtel"].split(" (")[0] if "www" in x["Propriété"] else x,
    axis=1,
)["Propriété"]
data["Propriété_clean"] = data["hôtel"].apply(lambda x: x.split(" (")[0])
data["extra_clean"] = data["extra"].apply(lambda x: x.split(" (")[0])
data["periode_fin"] = data["date_fin"].dt.strftime("%m-%Y")
data["Année"] = data["date_fin"].dt.year.astype(str)
data["Mois"] = data["date_fin"].dt.to_period("M").astype(str)
data["Semaine"] = data["date_fin"].dt.to_period("W-Mon").astype(str)
data["marge"] = data.apply(lambda x: x["total HT"] - x["montant HT"], axis=1)
data["mois"] = data["date_fin"].dt.strftime("%m")

data["Mois_WY"] = pd.DatetimeIndex(data["date_fin"]).month
data["Jour"] = data["date_fin"].dt.to_period("D").astype(str)

aujd = datetime.datetime.now()
mois_auj = aujd.strftime("%Y-%m")
premier_jour_du_mois =  aujd.replace(day=1).strftime("%Y-%m-%d")
mois_précédent = (
    aujd.now()
    .replace(month=aujd.month - 1)
    .strftime("%Y-%m")
)
mois_auj_clean = aujd.strftime("%B %Y") #utile pour un affiche en format nom du mois année
mois_année_précédente = (
    aujd
    .replace(year=aujd.year - 1)
    .strftime("%Y-%m")
)
mois_année_précédente_clean = ( #utile pour un affiche en format nom du mois année
    aujd
    .replace(year=aujd.year - 1)
    .strftime("%B %Y")
)



def format_currency(value):
    return f"{value:,} €"

In [198]:
data["Mois2"] = pd.to_datetime(data["date_fin"],format='%m').dt.to_period('M')


In [199]:
data.head()

,Propriété,hôtel,extra,date,statuts,tarif urgence,tarif horaire,nbre d'heures,total HT,facture,...,extra_clean,periode_fin,Année,Mois,Semaine,marge,mois,Mois_WY,Jour,Mois2
0,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,2024-01-07,NaN,NaN,21.0,12hr 0min,252,NaN,...,Grégory Tshimini,01-2024,2024,2024-01,2024-01-02/2024-01-08,72,01,1,2024-01-07,2024-01
1,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,2024-01-06,NaN,NaN,20.0,9hr 30min,180,NaN,...,Mamadou Koma,01-2024,2024,2024-01,2024-01-02/2024-01-08,45,01,1,2024-01-07,2024-01
2,Touraine,Touraine (https://www.notion.so/Touraine-df4e7...,Mohamed Belbachir (https://www.notion.so/Moham...,2024-01-06,NaN,NaN,20.0,12hr 0min,240,NaN,...,Mohamed Belbachir,01-2024,2024,2024-01,2024-01-02/2024-01-08,60,01,1,2024-01-07,2024-01
3,Crystal,Crystal (https://www.notion.so/Crystal-1837fcb...,Grégory Tshimini (https://www.notion.so/Gr-gor...,2024-01-06,NaN,NaN,21.0,12hr 0min,252,NaN,...,Grégory Tshimini,01-2024,2024,2024-01,2024-01-02/2024-01-08,72,01,1,2024-01-06,2024-01
4,Milie rose,Milie Rose (https://www.notion.so/Milie-Rose-0...,Mamadou Koma (https://www.notion.so/Mamadou-Ko...,2024-01-05,NaN,NaN,20.0,9hr 30min,180,NaN,...,Mamadou Koma,01-2024,2024,2024-01,2024-01-02/2024-01-08,45,01,1,2024-01-06,2024-01


In [220]:
data_grouby_CA = data.groupby(["mois", "Mois", "Année"])["total HT"].sum().reset_index()
data_grouby_CA 

,mois,Mois,Année,total HT
0,01,2022-01,2022,16256
1,01,2023-01,2023,39811
2,01,2024-01,2024,3154
3,02,2022-02,2022,16886
4,02,2023-02,2023,40302
5,03,2022-03,2022,18728
6,03,2023-03,2023,37626
7,04,2022-04,2022,16064
8,04,2023-04,2023,38719
9,05,2022-05,2022,21040


In [225]:
data_grouby_CA = data.groupby(["mois", "Mois", "Année"])["total HT"].sum().reset_index()
mask = data_grouby_CA["total HT"] != 0
data_grouby_CA = data_grouby_CA[mask]
annees = data_grouby_CA["Année"].unique()


couleur_dict={}
for i, annee in enumerate(annees):    
    couleur = px.colors.qualitative.Set1[i]
    couleur_dict.update({annee:couleur})


couleurs_ordre = list(couleur_dict.values())

print (couleurs_ordre)
print (couleur_dict)



['rgb(228,26,28)', 'rgb(55,126,184)', 'rgb(77,175,74)', 'rgb(152,78,163)']
{'2022': 'rgb(228,26,28)', '2023': 'rgb(55,126,184)', '2024': 'rgb(77,175,74)', '2021': 'rgb(152,78,163)'}


In [230]:
data_grouby_CA = data_grouby_CA.groupby(["mois", "Mois", "Année"])["total HT"].sum().reset_index()
mask = data_grouby_CA["total HT"] != 0
data_grouby_CA = data_grouby_CA[mask]
annees = data_grouby_CA["Année"].unique()


couleur_dict={}
for i, annee in enumerate(annees):    
    couleur = px.colors.qualitative.Set1[i]
    couleur_dict.update({annee:couleur})


mask = data_grouby_CA["Mois"] <= mois_précédent
annees_prec = data_grouby_CA[mask]["Année"].unique()
couleurs_annees_prec = [couleur_dict.get(annee, "valeur_par_défaut") for annee in annees_prec]

couleurs_annees_prec


['rgb(228,26,28)', 'rgb(55,126,184)', 'rgb(152,78,163)']

In [251]:
data_grouby_CA = data_grouby_CA.groupby(["mois", "Mois", "Année"])["total HT"].sum().reset_index()
mask = data_grouby_CA["total HT"] != 0
data_grouby_CA = data_grouby_CA[mask]
annees = data_grouby_CA["Année"].unique()


couleur_dict={}
for i, annee in enumerate(annees):    
    couleur = px.colors.qualitative.Set1[i]
    couleur_dict.update({annee:couleur})


mask = data_grouby_CA["Mois"] <= mois_précédent
annees_prec = data_grouby_CA[mask]["Année"].unique()
couleurs_annees_prec = [couleur_dict.get(annee, "valeur_par_défaut") for annee in annees_prec]

fig = px.line(
    data_grouby_CA[mask],
    x="mois",
    y="total HT",
    text = data_grouby_CA[mask]["total HT"].map(format_currency),
    color="Année",  
    labels={"total HT": "CA", "mois": "Mois"},
    title="Évolution du chiffre d'affaires par mois",
    color_discrete_sequence=couleurs_annees_prec,
)

fig.update_layout(
    legend=dict(traceorder='reversed'),
)

fig.update_layout(
    xaxis=dict(categoryorder="array", categoryarray=sorted(data_grouby_CA[mask]["mois"].unique()))
)


for i, annee in enumerate(annees):
    mask = (data_grouby_CA["Mois"] >= mois_précédent) & (data_grouby_CA["Année"] == annee)
    color = couleur_dict[annee]
    fig.add_trace(
        go.Scatter(name = f"En cours pour {annee}",
            x=data_grouby_CA[mask]["mois"],
            y=data_grouby_CA[mask]["total HT"],
            mode="lines+markers+text",
            line=dict(color=color, width=2, dash="dash"),
            text=data_grouby_CA[mask]["total HT"].map(format_currency),
            textposition="top center",
        )
    )


#fig.for_each_trace(lambda trace : trace.update({"line":couleur_dict)})


fig.update_traces(textposition="bottom center")
fig.update_xaxes(type="category")
fig.update_yaxes(title_text="CA en k€")

